# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Perform an ETL process with multiple tables and create a single dataset

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this SQL query to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
```

In [1]:
# Your code here
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('lego.db')
cur = conn.cursor()

In [5]:
def Query(string):
    cur.execute(string)
    df = pd.DataFrame(cur.fetchall())
    df.columns = [i[0] for i in cur.description]
    return df

In [8]:
tables = Query('''SELECT name FROM sqlite_master
                           WHERE type='table'
                           ORDER BY name;''')
tables

,name
0,product_details
1,product_info
2,product_pricing
3,product_reviews


In [14]:
Query(f'''SELECT COUNT(*) FROM product_details''')['COUNT(*)']

0    744
Name: COUNT(*), dtype: int64

In [16]:
for table in list(tables['name']):
    length = list(Query(f'''SELECT COUNT(*) as num_entries FROM {table}''')['num_entries'])[0]
    results = Query(f'''SELECT * FROM {table} LIMIT 5''')
    
    print(table, 'has', length, 'entries', '\n', results, '\n')

product_details has 744 entries 
    prod_id prod_desc prod_long_desc theme_name
0      630      blah       blahblah    Classic
1     2304      blah       blahblah     DUPLO®
2     7280      blah       blahblah       City
3     7281      blah       blahblah       City
4     7499      blah       blahblah       City 

product_info has 744 entries 
    prod_id  ages  piece_count                         set_name
0      630    4+            1                  Brick Separator
1     2304  1½-5            1     LEGO® DUPLO® Green Baseplate
2     7280  5-12            2      Straight & Crossroad Plates
3     7281  5-12            2  T-Junction & Curved Road Plates
4     7499  5-12           24     Flexible and Straight Tracks 

product_pricing has 10870 entries 
    prod_id country list_price
0    75823      US      29.99
1    75822      US      19.99
2    75821      US      12.99
3    21030      US      99.99
4    21035      US      79.99 

product_reviews has 744 entries 
    prod_id  num_rev

In [18]:
combined_df = Query('''SELECT * FROM product_details 
                                JOIN product_info
                                USING(prod_id)
                                JOIN product_pricing
                                USING(prod_id)
                                JOIN product_reviews
                                USING(prod_id);''')

print(len(combined_df))
combined_df.head(10)

10870


,prod_id,prod_desc,prod_long_desc,theme_name,ages,piece_count,set_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,blah,blahblah,Angry Birds™,6-12,277,Bird Island Egg Heist,US,29.99,2.0,4.0,Average,4.5,4.0
1,75822,blah,blahblah,Angry Birds™,6-12,168,Piggy Plane Attack,US,19.99,2.0,4.0,Easy,5.0,4.0
2,75821,blah,blahblah,Angry Birds™,6-12,74,Piggy Car Escape,US,12.99,11.0,4.3,Easy,4.3,4.1
3,21030,blah,blahblah,Architecture,12+,1032,United States Capitol Building,US,99.99,23.0,3.6,Average,4.6,4.3
4,21035,blah,blahblah,Architecture,12+,744,Solomon R. Guggenheim Museum®,US,79.99,14.0,3.2,Challenging,4.6,4.1
5,21039,blah,blahblah,Architecture,12+,597,Shanghai,US,59.99,7.0,3.7,Average,4.9,4.4
6,21028,blah,blahblah,Architecture,12+,598,New York City,US,59.99,37.0,3.7,Average,4.2,4.1
7,21029,blah,blahblah,Architecture,12+,780,Buckingham Palace,US,49.99,24.0,4.4,Average,4.7,4.3
8,21034,blah,blahblah,Architecture,12+,468,London,US,39.99,23.0,3.6,Average,4.7,4.1
9,21033,blah,blahblah,Architecture,12+,444,Chicago,US,39.99,11.0,3.6,Average,4.8,4.5


In [21]:
combined_df['list_price'] = pd.to_numeric(combined_df['list_price'])

In [22]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
prod_desc            10870 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
country              10870 non-null object
list_price           10870 non-null float64
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    10870 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(5), int64(2), object(7)
memory usage: 1.2+ MB


In [23]:
combined_df.to_csv('LEGO_data_merged.csv', index = False)

## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity! In this lab, you successfully executed an ETL process to merge different tables!